# Make Diagnostic Plots for NA-CORDEX Zarr Stores

In [3]:
import xarray as xr
import numpy as np
import intake
import ast

import matplotlib.pyplot as plt

## Create and Connect to a Dask Distributed Cluster

In [ ]:
import dask
from ncar_jobqueue import NCARCluster

num_jobs = 8
walltime = "0:20:00"
memory='6GB' 
#cluster = NCARCluster(cores=num_jobs, processes=1, memory=memory, walltime=walltime)
#cluster.scale(jobs=num_jobs)
cluster = NCARCluster()

from distributed import Client
client = Client(cluster)
cluster

☝️ Link to Dask dashboard will appear above.

## Find and Obtain Data Using an Intake Catalog

#### Open catalog with list entries in "model" and produce a content summary

In [8]:
# Open collection description file
#catalog_url = "https://raw.githubusercontent.com/bonnland/na-cordex-aws/version_1.0/catalogs/aws-na-cordex.json"
catalog_url = "../catalogs/aws-na-cordex.json"
col = intake.open_esm_datastore(catalog_url, csv_kwargs={"converters": {"model": ast.literal_eval}},)
col

,unique
long_name,18
units,10
standard_name,10
spatial_domain,1
vertical_levels,1
start_time,3
end_time,4
model,9
frequency,1
variable,15


#### Open exploded catalog and produce a content summary

In [8]:
# Open collection description file
catalog_url = "https://raw.githubusercontent.com/bonnland/na-cordex-aws/version_1.0/catalogs/aws-na-cordex.json"
col = intake.open_esm_datastore(catalog_url)
col

,unique
long_name,18
units,10
standard_name,10
spatial_domain,1
vertical_levels,1
start_time,3
end_time,4
model,26
frequency,1
variable,15


In [5]:
# Show the first few lines of the catalog
col.df.head(10)

,long_name,units,standard_name,spatial_domain,vertical_levels,start_time,end_time,model,frequency,variable,scenario,grid,bias_correction,spatial_resolution,path
0,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1979-01-01T12:00:00,2014-12-31T12:00:00,['ERA-Int.CRCM5-UQAM' 'ERA-Int.CRCM5-OUR' 'ERA...,day,hurs,eval,NAM-22i,raw,0.25 deg,s3://ncar-na-cordex/day/hurs.eval.day.NAM-22i....
1,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1979-01-01T12:00:00,2015-12-31T12:00:00,['ERA-Int.CRCM5-UQAM' 'ERA-Int.RegCM4' 'ERA-In...,day,hurs,eval,NAM-44i,raw,0.5 deg,s3://ncar-na-cordex/day/hurs.eval.day.NAM-44i....
2,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['CanESM2.CanRCM4'],day,hurs,hist-rcp45,NAM-22i,mbcn-Daymet,0.25 deg,s3://ncar-na-cordex/day/hurs.hist-rcp45.day.NA...
3,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['CanESM2.CanRCM4'],day,hurs,hist-rcp45,NAM-22i,mbcn-gridMET,0.25 deg,s3://ncar-na-cordex/day/hurs.hist-rcp45.day.NA...
4,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['GFDL-ESM2M.CRCM5-OUR' 'CanESM2.CRCM5-OUR' 'C...,day,hurs,hist-rcp45,NAM-22i,raw,0.25 deg,s3://ncar-na-cordex/day/hurs.hist-rcp45.day.NA...
5,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['MPI-ESM-LR.CRCM5-UQAM' 'CanESM2.CRCM5-UQAM' ...,day,hurs,hist-rcp45,NAM-44i,mbcn-Daymet,0.5 deg,s3://ncar-na-cordex/day/hurs.hist-rcp45.day.NA...
6,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['MPI-ESM-LR.CRCM5-UQAM' 'CanESM2.CRCM5-UQAM' ...,day,hurs,hist-rcp45,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/hurs.hist-rcp45.day.NA...
7,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['MPI-ESM-LR.CRCM5-UQAM' 'CanESM2.CRCM5-UQAM' ...,day,hurs,hist-rcp45,NAM-44i,raw,0.5 deg,s3://ncar-na-cordex/day/hurs.hist-rcp45.day.NA...
8,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['MPI-ESM-MR.CRCM5-UQAM' 'GEMatm-Can.CRCM5-UQA...,day,hurs,hist-rcp85,NAM-22i,mbcn-Daymet,0.25 deg,s3://ncar-na-cordex/day/hurs.hist-rcp85.day.NA...
9,Near-Surface Relative Humidity,%,relative_humidity,north_america,1,1949-01-01T12:00:00,2100-12-31T12:00:00,['MPI-ESM-MR.CRCM5-UQAM' 'GEMatm-Can.CRCM5-UQA...,day,hurs,hist-rcp85,NAM-22i,mbcn-gridMET,0.25 deg,s3://ncar-na-cordex/day/hurs.hist-rcp85.day.NA...


In [10]:
# Show expanded version of collection structure with details
import pprint

uniques = col.unique(
    columns=["variable", "scenario", "grid", "model", "bias_correction"]
)
pprint.pprint(uniques, compact=True, indent=4)

{   'bias_correction': {   'count': 3,
                           'values': ['raw', 'mbcn-Daymet', 'mbcn-gridMET']},
    'grid': {'count': 2, 'values': ['NAM-22i', 'NAM-44i']},
    'model': {   'count': 26,
                 'values': [   'EC-EARTH.RCA4', 'GFDL-ESM2M.RegCM4',
                               'ERA-Int.RegCM4', 'CanESM2.CanRCM4',
                               'EC-EARTH.HIRHAM5', 'CanESM2.RCA4',
                               'HadGEM2-ES.WRF', 'ERA-Int.WRF',
                               'ERA-Int.CRCM5-UQAM', 'GEMatm-MPI.CRCM5-UQAM',
                               'ERA-Int.RCA4', 'ERA-Int.HIRHAM5',
                               'MPI-ESM-LR.RegCM4', 'GFDL-ESM2M.WRF',
                               'MPI-ESM-MR.CRCM5-UQAM', 'MPI-ESM-LR.WRF',
                               'MPI-ESM-LR.CRCM5-OUR', 'HadGEM2-ES.RegCM4',
                               'ERA-Int.CanRCM4', 'CanESM2.CRCM5-OUR',
                               'GEMatm-Can.CRCM5-UQAM', 'CanESM2.CRCM5-UQAM',
            

#### Load data into xarray using the catalog

In [13]:
data_var = 'tmin'

col_subset = col.search(
    variable=data_var,
    grid="NAM-44i",
    scenario="hist",
    bias_correction="mbcn-gridMET",
)

col_subset

,unique
long_name,1
units,1
standard_name,1
spatial_domain,1
vertical_levels,1
start_time,1
end_time,1
model,15
frequency,1
variable,1


In [14]:
col_subset.df

,long_name,units,standard_name,spatial_domain,vertical_levels,start_time,end_time,model,frequency,variable,scenario,grid,bias_correction,spatial_resolution,path
0,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,MPI-ESM-MR.CRCM5-UQAM,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
1,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,GEMatm-Can.CRCM5-UQAM,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
2,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,MPI-ESM-LR.CRCM5-UQAM,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
3,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,CanESM2.CRCM5-UQAM,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
4,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,GEMatm-MPI.CRCM5-UQAM,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
5,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,HadGEM2-ES.RegCM4,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
6,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,GFDL-ESM2M.RegCM4,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
7,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,MPI-ESM-LR.RegCM4,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
8,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,EC-EARTH.HIRHAM5,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....
9,Daily Minimum Near-Surface Air Temperature (Bi...,degC,air_temperature,north_america,1,1949-01-01T12:00:00,2005-12-31T12:00:00,EC-EARTH.RCA4,day,tmin,hist,NAM-44i,mbcn-gridMET,0.5 deg,s3://ncar-na-cordex/day/tmin.hist.day.NAM-44i....


In [15]:
# Load catalog entries for subset into a dictionary of xarray datasets
dsets = col_subset.to_dataset_dict(
    zarr_kwargs={"consolidated": True}, storage_options={"anon": True}
)
print(f"\nDataset dictionary keys:\n {dsets.keys()}")

ds = dsets["day.hist.NAM-44i.mbcn-gridMET"]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'frequency.scenario.grid.bias_correction'



Dataset dictionary keys:
 dict_keys(['day.hist.NAM-44i.mbcn-gridMET'])


## Functions for Plotting

#### Helper Function to Create a Single Map Plot

In [17]:
def plotMap(ax, map_slice, date_object=None, member_id=None):
    """Create a map plot on the given axes, with min/max as text"""

    ax.imshow(map_slice, origin='lower')

    minval = map_slice.min(dim = ['lat', 'lon'])
    maxval = map_slice.max(dim = ['lat', 'lon'])

    # Format values to have at least 4 digits of precision.
    ax.text(0.01, 0.03, "Min: %3g" % minval, transform=ax.transAxes, fontsize=12)
    ax.text(0.99, 0.03, "Max: %3g" % maxval, transform=ax.transAxes, fontsize=12, horizontalalignment='right')
    ax.set_xticks([])
    ax.set_yticks([])
    
    if date_object:
        ax.set_title(date_object.values.astype(str)[:10], fontsize=12)
        
    if member_id:
        ax.set_ylabel(member_id, fontsize=12)
        
    return ax

#### Helper Function for Finding Dates with Available Data

In [18]:
def getValidDateIndexes(member_slice):
    """Search for the first and last dates with finite values."""
    min_values = member_slice.min(dim = ['lat', 'lon'])
    is_finite = np.isfinite(min_values)
    finite_indexes = np.where(is_finite)

    start_index = finite_indexes[0][0]
    end_index = finite_indexes[0][-1]
    return start_index, end_index

#### Function Producing Maps of First, Middle, and Final Timesteps

In [19]:
def plot_first_mid_last(ds, data_var):
    """Plot the first, middle, and final time steps for several climate runs."""
    numEnsembleMembers = 4
    member_names = ds.coords['member_id'].values[0:numEnsembleMembers]
    
    figWidth = 18 
    figHeight = 12 
    numPlotColumns = 3
    fig, axs = plt.subplots(numEnsembleMembers, numPlotColumns, figsize=(figWidth, figHeight), constrained_layout=True)

    for index in np.arange(numEnsembleMembers):
        mem_id = member_names[index]
        data_slice = ds[data_var].sel(member_id=mem_id)
           
        start_index, end_index = getValidDateIndexes(data_slice)
        midDateIndex = np.floor(len(ds.time) / 2).astype(int)

        startDate = ds.time[start_index]
        first_step = data_slice.sel(time=startDate) 
        ax = axs[index, 0]
        plotMap(ax, first_step, startDate, mem_id)

        midDate = ds.time[midDateIndex]
        mid_step = data_slice.sel(time=midDate)   
        ax = axs[index, 1]
        plotMap(ax, mid_step, midDate)

        endDate = ds.time[end_index]
        last_step = data_slice.sel(time=endDate)            
        ax = axs[index, 2]
        plotMap(ax, last_step, endDate)
             
    plt.show()

#### Function Producing Time Series Plots
Also show which dates have no available data values.

In [20]:
def plot_timeseries(ds, data_var):
    """Plot the mean, min, max, and standard deviation values for several climate runs."""

    numEnsembleMembers = 4
    member_names = ds.coords['member_id'].values[0:numEnsembleMembers]

    figWidth = 25 
    figHeight = 20
    linewidth = 0.5

    numPlotColumns = 1
    fig, axs = plt.subplots(numEnsembleMembers, numPlotColumns, figsize=(figWidth, figHeight))
        
    for index in np.arange(numEnsembleMembers):
        mem_id = member_names[index]
        data_slice = ds[data_var].sel(member_id=mem_id)
        unit_string = ds[data_var].attrs['units']
        
        min_vals = data_slice.min(dim = ['lat', 'lon'])
        max_vals = data_slice.max(dim = ['lat', 'lon'])
        mean_vals = data_slice.mean(dim = ['lat', 'lon'])
        std_vals = data_slice.std(dim = ['lat', 'lon'])

        missing_indexes = np.isnan(min_vals)
        missing_times = ds.time[nan_indexes]

        axs[index].plot(ds.time, min_vals, linewidth=linewidth, label='min')
        axs[index].plot(ds.time, max_vals, linewidth=linewidth, label='max')
        axs[index].plot(ds.time, mean_vals, linewidth=linewidth, label='mean')
        axs[index].plot(ds.time, std_vals, linewidth=linewidth, label='std')
            
        ymin, ymax = axs[index].get_ylim()
        rug_y = ymin + 0.01*(ymax-ymin)
        axs[index].plot(missing_times, [rug_y]*len(missing_times), '|', color='m', label='missing')
        axs[index].set_title(mem_id, fontsize=20)
        axs[index].legend(loc='upper right')
        axs[index].set_ylabel(unit_string)

        plt.tight_layout(pad=10.2, w_pad=3.5, h_pad=3.5)
        
    plt.show()

## Produce Diagnostic Plots

#### Plot First, Middle, and Final Timesteps for several output runs (less data intensive)

In [ ]:
%%time

# Plot using the Zarr Store obtained from an earlier step in the notebook.
plot_first_mid_last(ds, data_var)

#### Plot Time Series for several output runs (more data intensive)

In [ ]:
%%time

# Plot using the Zarr Store obtained from an earlier step in the notebook.
plot_timeseries(ds, data_var)

### Release the workers.

In [ ]:
!date

In [ ]:
cluster.close()